In [2]:
import sys
sys.path.insert(0, '/content/drive/MyDrive/Question 5 - ML/src')
sys.path.insert(0, '/content/drive/MyDrive/Question 5 - ML/pipelines')


import pandas as pd
import joblib



def load_model_and_predict(store, item_dept, date, history_file):
    # Load the model from disk
    model = joblib.load('/content/drive/MyDrive/Question 5 - ML/pipelines/xgboost_model.pkl')

    # Load historical data from CSV with date parsing
    history = pd.read_csv(history_file, parse_dates=['date_id'], index_col='date_id')

    # Ensure the date is a datetime object
    date = pd.to_datetime(date)

    # Filter historical data for the specific store, department, and date
    input_data = history[(history['store'] == store) &
                         (history['item_dept'] == item_dept) &
                         (history.index.date == date.date())]


    # Select and format the features required by the model
    input_features = input_data[['item_qty_lag_1', 'item_qty_lag_7', 'net_sales_lag_1', 'net_sales_lag_7',
                                 'day_of_week', 'is_weekend', 'month', 'quarter', 'year',
                                 'store_sales_ratio', 'price_per_item']]

    predicted_qty = model.predict(input_features)
    print(f"Predicted item quantity for {store} - {item_dept} on {date}: {predicted_qty[0]}")


In [3]:
def main():
    st.title('Item Quantity Prediction')

    store = st.text_input('Store')
    item_dept = st.text_input('Item Department')
    date = st.date_input('Date', datetime.now())
    history_file = st.file_uploader('Upload historical data CSV', type=['csv'])

    if st.button('Predict'):
        if history_file is not None:
            prediction = load_model_and_predict(store, item_dept, date, history_file)
            st.write(f"Predicted item quantity for {store} - {item_dept} on {date}: {prediction}")
        else:
            st.error('Please upload a historical data CSV file.')
